In [11]:
import os
import numpy as np

rootDir, subsetList, _ = next(os.walk("/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/feats_v05/valid"))

tmp = np.load(os.path.join(rootDir, 'feat_1', 'valid_0_1.npy'))
cumsum = np.zeros_like(tmp)

for subdir in sorted(subsetList):
    if subdir == 'feat_x':
        continue
    cumsum += np.load(os.path.join(rootDir, subdir, 'valid_0_1.npy'))
    print(subdir)
    
mean = cumsum / (len(subsetList)-1)
    
np.save(os.path.join(rootDir, 'feat_x', 'valid_0_1.npy'), mean)

feat_1
feat_10
feat_100
feat_1000
feat_1001
feat_1002
feat_1003
feat_1004
feat_1005
feat_1006
feat_1007
feat_1008
feat_101
feat_102
feat_103
feat_104
feat_105
feat_106
feat_107
feat_108
feat_109
feat_11
feat_110
feat_111
feat_112
feat_113
feat_114
feat_115
feat_116
feat_117
feat_118
feat_119
feat_12
feat_120
feat_121
feat_122
feat_123
feat_124
feat_125
feat_126
feat_127
feat_128
feat_129
feat_13
feat_130
feat_131
feat_132
feat_133
feat_134
feat_135
feat_136
feat_137
feat_138
feat_139
feat_14
feat_140
feat_141
feat_142
feat_143
feat_144
feat_145
feat_146
feat_147
feat_148
feat_149
feat_15
feat_150
feat_151
feat_152
feat_153
feat_154
feat_155
feat_156
feat_157
feat_158
feat_159
feat_16
feat_160
feat_161
feat_162
feat_163
feat_164
feat_165
feat_166
feat_167
feat_168
feat_169
feat_17
feat_170
feat_171
feat_172
feat_173
feat_174
feat_175
feat_176
feat_177
feat_178
feat_179
feat_18
feat_180
feat_181
feat_182
feat_183
feat_184
feat_185
feat_186
feat_187
feat_188
feat_189
feat_19
feat_190
feat

In [8]:
tmp.shape

(1888037, 768)

In [6]:
import os
import numpy as np

rootDir, subsetList, _ = next(os.walk("/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/feats_v05/valid"))

full = list(np.arange(1,1009))
partial = []
for subdir in sorted(subsetList):
    idx = subdir.split('_')[1]
    partial.append(int(idx))
    
set(full)-set(partial)

{338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378}

In [1]:
import fairseq
import torch
import soundfile as sf
import joblib

device = 'cuda:0'

ckpt_path = "/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/hubert_base_ls960.pt"
models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path], 
                                                                          {'extractor_mode': 'group_norm_masked'},
                                                                          strict=True)
model = models[0].eval().to(device)

2021-12-17 22:34:22 | INFO | fairseq.tasks.hubert_pretraining | current directory is /gpfs/u/scratch/LANG/LANGkzhq/fairseq/examples/hubert/simple_kmeans
2021-12-17 22:34:22 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/wnhsu/data/librispeech/960h/iter/250K_50hz_km100_mp0_65_v2', 'fine_tuning': False, 'labels': ['layer6.km500'], 'label_dir': None, 'label_rate': 50, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False, 'spk2info': None}
2021-12-17 22:34:22 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50, 'extractor_mode': group_norm_masked, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'dropout': 0.1, 'attention_dropout': 0.

In [5]:
from fairseq.data.data_utils import lengths_to_padding_mask
import numpy as np
import soundfile as sf

x1, fs = sf.read("/gpfs/u/scratch/LANG/shared/data/Librispeech/train-clean-100/103/1240/103-1240-0000.flac")
x2, fs = sf.read("/gpfs/u/scratch/LANG/shared/data/Librispeech/train-clean-100/103/1240/103-1240-0001.flac")
x3, fs = sf.read("/gpfs/u/scratch/LANG/shared/data/Librispeech/train-clean-100/103/1240/103-1240-0002.flac")
x4, fs = sf.read("/gpfs/u/scratch/LANG/shared/data/Librispeech/train-clean-100/103/1240/103-1240-0003.flac")

l1, l2, l3, l4 = len(x1), len(x2), len(x3), len(x4)
lengths = torch.tensor([l1,l2,l3,l4], device=device)
l_max = lengths.max().item()
mask = lengths_to_padding_mask(lengths)

x1p = np.pad(x1, (0, l_max-l1))
x2p = np.pad(x2, (0, l_max-l2))
x3p = np.pad(x3, (0, l_max-l3))
x4p = np.pad(x4, (0, l_max-l4))

xB = torch.from_numpy(np.stack([x1p,x2p,x3p,x4p], axis=0)).float().to(device)
x1 = torch.from_numpy(x1).unsqueeze(0).float().to(device)
x2 = torch.from_numpy(x2).unsqueeze(0).float().to(device)
x3 = torch.from_numpy(x3).unsqueeze(0).float().to(device)
x4 = torch.from_numpy(x4).unsqueeze(0).float().to(device)

In [6]:
with torch.no_grad():
    featB, _ = model.extract_features(
        source=xB,
        padding_mask=mask,
        mask=False,
        output_layer=12,
    )
    
    feat1, _ = model.extract_features(
        source=x1,
        padding_mask=None,
        mask=False,
        output_layer=12,
    )
    
    feat2, _ = model.extract_features(
        source=x2,
        padding_mask=None,
        mask=False,
        output_layer=12,
    )
    
    feat3, _ = model.extract_features(
        source=x3,
        padding_mask=None,
        mask=False,
        output_layer=12,
    )
    
    feat4, _ = model.extract_features(
        source=x4,
        padding_mask=None,
        mask=False,
        output_layer=12,
    )

In [7]:
(featB[2, :feat3.size(1), :] - feat3[0, :, :]).abs().max()

tensor(2.1219e-05, device='cuda:0')

In [10]:
# remove junk files
from time import time
import numpy as np
import os
import shutil

rootDir, subsetList, _ = next(os.walk("/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/feats_v05/valid"))

for subdir in sorted(subsetList):
    try:
        tmp = np.load(os.path.join(rootDir, subdir, 'valid_0_1.npy'), mmap_mode='r')
    except:
        print(f'noexist {subdir}')
        #shutil.rmtree(os.path.join(rootDir, subdir))
    if tmp.shape != (1888037, 768):
        print(f'partial {subdir}')
        #shutil.rmtree(os.path.join(rootDir, subdir))

noexist feat_337
noexist feat_x


In [5]:
# account
from time import time
import numpy as np
import os
import shutil

tot_len = 172812419
step = tot_len // 16

rootDir, subsetList, _ = next(os.walk("/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/feats_v05/train"))

cumsum = np.zeros((step, 768), dtype=np.float32)

for subdir in sorted(subsetList):
    tic = time()
    tmp = np.load(os.path.join(rootDir, subdir, 'train_0_1.npy'), mmap_mode='r')
    cumsum += tmp[step:2*step, :]
    print(subdir, time()-tic)

feat_0 17.222425937652588
feat_1 17.39583468437195
feat_10 33.66486144065857
feat_100 17.306034803390503
feat_1000 17.24469566345215
feat_1001 17.0453622341156
feat_1003 17.43369722366333
feat_1004 17.37086057662964
feat_1006 17.20004963874817
feat_1007 17.28095316886902
feat_1009 17.288715839385986
feat_101 17.30957531929016
feat_1010 17.341358184814453
feat_1012 17.15835213661194
feat_1013 17.203232288360596
feat_1015 16.99709129333496
feat_1016 17.177743434906006
feat_1027 17.126038074493408
feat_1028 17.26115870475769
feat_103 16.963723182678223
feat_1030 17.261507511138916
feat_1031 17.27083396911621
feat_1033 17.155521631240845
feat_1034 17.181316137313843
feat_1036 17.359928846359253
feat_1037 17.2775239944458
feat_1039 17.168633699417114
feat_104 17.1607825756073
feat_1040 17.076789617538452
feat_1042 17.110933303833008
feat_1043 17.778125047683716
feat_1045 16.906445503234863
feat_1046 17.46548342704773
feat_1048 17.61017107963562
feat_1049 17.258878469467163
feat_1051 17.0392

KeyboardInterrupt: 

In [2]:
np.save(os.path.join(rootDir, 'feat_x', 'test_0_1.npy'), cumsum)

In [5]:
tic = time()
np.load(os.path.join(rootDir, 'feat_x', 'test_0_1.npy'))
print(subdir, time()-tic)

feat_1058 22.451002597808838


In [9]:
tmp.shape

(172812419, 768)

In [40]:
from time import time
import numpy as np
import os
import shutil
import sys

k = 2

partition = []
tot_len = 172812419
step = 5400388

for i in range(0, tot_len-step, step):
    partition.append([i, min(i+step, tot_len)])
partition[-1][1] = tot_len

rootDir, subsetList, _ = next(os.walk("/gpfs/u/scratch/LANG/LANGkzhq/ssl-disentangle/feats_v05/train"))

cumsum = np.zeros((step, 768), dtype=np.float32)

for subdir in sorted(subsetList):
    tmp = np.load(os.path.join(rootDir, subdir, 'train_0_1.npy'), mmap_mode='r')
    cumsum += tmp[partition[k][0]:partition[k][1], :]
    print(subdir)

mean = cumsum / len(subsetList)

np.save(os.path.join(rootDir, 'feat_x', f'train_{k}_24.npy'), mean)

In [44]:
partition = []
tot_len = 172812419
step = 5400388

for i in range(0, tot_len-step, step):
    partition.append([i, min(i+step, tot_len)])
partition[-1][1] = tot_len

In [45]:
len(partition)

32